# DocumentReference factory

create a bunch of DopcumentReference objects from a csv file and save them to a folder and create Bundle objects from the conditions and save them to a folder

1. Load DocumentReference Template
2. get csv in form

|DocumentReference #|_id|_datetime|_code|_file|
|---|---|---|---|---|
|Foo|foo|2023-08-21T20:16:25.019Z|12345678|Examplitis|

4. Convert csv to DataFrame
<!-- 2. Create a transaction Bundle
1. Load Patient into Bundle
2. Load the data row into the DocumentReference template
3. Add to the Bundler -->
5. Create DocumentReference Instance
5. Validate 

### Define Globals and Imports

In [8]:
from hashlib import sha1
import base64
from pathlib import Path
from IPython.display import display, Markdown as md, HTML
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources.documentreference import DocumentReference
from json import dumps, loads
import pandas as pd
from requests import get, post
from datetime import datetime
from dateutil import tz
import mimetypes

in_path = Path.cwd() / 'source_data' /'docref.csv'
out_path = Path.cwd() /'examples'
print(in_path)
print(out_path)
# _system = 'http://snomed.info/sct'
_system = 'http://loinc.org'
fhir_test_server = 'https://r4.ontoserver.csiro.au/fhir'

patient_id = 'Patent/example' #'cdex-example-patient'
author_id = 'Practitioner/practitioner-1' #'cdex-example-practitioner'
_now = datetime.now(tz=tz.tzlocal()).isoformat()
print(_now)

authentication_time_extension = {
    #'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-authentication-time',
    'url': 'http://hl7.org/fhir/us/uscdi5-sandbox/StructureDefinition/us-core-authentication-time',
    'valueDateTime': _now
    }

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json',
}
# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient


/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/source_data/docref.csv
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/examples
2024-10-08T19:48:54.316108-07:00


In [9]:
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/PatientHealthQuestionnaire-2_v1.0_2014Jul2.pdf")
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/AHCM-HealthSocialNeedsScreeningTool-rev-8_10_2021.pdf")
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/Example3PRAPARE-English.pdf")

### Load Docref Template

In [10]:
# title ='Example3PRAPAREQuestionnaire' #'Example2AHN-HRSNQuestionnaire' #'Example1PHQ-2Questionnaire'
docref  = DocumentReference.parse_file('/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/QR-OBs-DE/docref.yml')
my_cat = docref.category


# docref.subject.reference = f'Patient/{patient_i}'
# docref.author[0].reference = f'Practitioner/{author_id}'
print(f'docref.content[0].attachment.data = {docref.content[0].attachment.data}')
print(f'type= {type(docref.content[0].attachment.data)}')

docref.content[0].attachment.data = b'JVBERi0xLjQKJ...'
type= <class 'bytes'>


###  Get CSV File ( hard coded for now) and Convert to DF

In [11]:
from io import StringIO  # for hardcoded csv only
# my_csv = '''_title,_id,_datetime,_code,_file
# Progress Note,progress-note,2023-08-21T20:16:25.019Z,11506-3,/Users/ehaas/Downloads/progress-note.pdf
# Imaging Study,imaging-study,2021-01-21,18748-4,/Users/ehaas/Downloads/imaging-study.pdf
# Surgical Note,surgical-note,2018-02-02,11504-8,/Users/ehaas/Downloads/surgical-note.pdf
# CCD,ccd,2023-08-22,34133-9,/Users/ehaas/Downloads/example-1-ccd.xml
# History and Physical Notes,history-and-physical-notes,2024-07-16T11:31:17Z,34117-2,/Users/ehaas/Pictures/icons/vectorstock_31051226/vectorstock_31051226.pdf
# Referral Notes,referral-notes,2024-07-16T11:31:17Z,57133-1,/Users/ehaas/Pictures/icons/vectorstock_31051226/vectorstock_31051226.pdf
# Consultation Notes,consultation-notes,2024-07-16T11:31:17Z,11488-4,/Users/ehaas/Pictures/icons/vectorstock_31051226/vectorstock_31051226.pdf
# Procedure Notes,procedure-notes,2024-07-16T11:31:17Z,28570-0,/Users/ehaas/Pictures/icons/vectorstock_31051226/vectorstock_31051226.pdf
# Care Plans,care-plans,2024-07-16T11:31:17Z,18776-5,/Users/ehaas/Pictures/icons/vectorstock_31051226/vectorstock_31051226.pdf
# '''

my_csv = Path(r"/Users/ehaas/Documents/FHIR/davinci-ecdx/my-notes/202409Connectathon/attachment_example_files.csv")
df = pd.read_csv(my_csv, sep=",", dtype='str')
df.dropna(how="all", inplace=True) # remove empty lines
df

,_title,_id,_datetime,_code,_category,_file,_subject,_author,_authenticator,_profile,_profile_title
0,Progress Note,progress-note,2023-08-21T20:16:25.019Z,11506-3,NaN,/Users/ehaas/Downloads/progress-note.pdf,Patient/example,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
1,Imaging Study (PDF),imaging-study-pdf,2024-10-04,18748-4,NaN,/Users/ehaas/Downloads/imaging-study.pdf,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
2,Surgical Note,surgical-note,2024-10-04,11504-8,NaN,/Users/ehaas/Downloads/surgical-note.pdf,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
3,CCD,ccd,2024-10-04,34133-9,NaN,/Users/ehaas/Downloads/example-1-ccd.xml,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
4,History and Physical Notes,history-and-physical-notes,2024-07-16T11:31:17Z,34117-2,NaN,/Users/ehaas/Pictures/icons/vectorstock_310512...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
5,Referral Notes,referral-notes,2024-07-16T11:31:17Z,57133-1,NaN,/Users/ehaas/Pictures/icons/vectorstock_310512...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
6,Consultation Notes,consultation-notes,2024-07-16T11:31:17Z,11488-4,NaN,/Users/ehaas/Pictures/icons/vectorstock_310512...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
7,Procedure Notes,procedure-notes,2024-07-16T11:31:17Z,28570-0,NaN,/Users/ehaas/Pictures/icons/vectorstock_310512...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
8,Care Plans,care-plans,2024-07-16T11:31:17Z,18776-5,NaN,/Users/ehaas/Pictures/icons/vectorstock_310512...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN
9,Imaging Study (JPEG),imaging-study-jpeg,2024-09-18T17:45:42Z,18748-4,NaN,/Users/ehaas/Downloads/640px-Cat_August_2010-4...,Patient/cdex-example-patient,Practitioner/cdex-example-practitioner,Practitioner/cdex-example-practitioner,NaN,NaN


### Create DocRef Instance

1. Get Data Element row by row from DF

In [12]:
def get_display(_system,_code,_version=None):
    """for example Looking up a code/system
       GET [base]/CodeSystem/$lookup?system=http://loinc.org&code=1963-8
    """
    params = dict(
       system = _system,
      # system = 'http://loinc.org',
      code = _code,
      # code = '1963-8'
        )
    r = get(f'{fhir_test_server}/CodeSystem/$lookup', params = params, headers = headers)
    print(r.status_code, r.reason)
    print(r.json())
    if r.status_code != 200:
        return 'examplitis'
    else:
      return r.json()['parameter'][1]['valueString']

In [13]:
def get_binary(file_path):
    """get binary from file path"""
    with open(file_path, 'rb') as f:
        binary = base64.b64encode(f.read()).decode('utf-8')  # return string
    return binary

In [14]:
for i in range(len(df)):
    print(i)
    _title,_id,_datetime,_code,_category,_file,_subject,_author,_authenticator,_profile, _profile_title = df.iloc[i]
    print(_title)
    # docref.meta.extension = None  #remove title and description
    if _profile == _profile:
        docref.meta.profile[0] = _profile
        docref.meta.extension[0].valueString = f'DocRef {_title}'
        docref.meta.extension[1].valueMarkdown = f'This is a an example of {_title} for the *{_profile_title}*'
    docref.meta.extension[1].valueMarkdown
    code_display = get_display(_system,_code)
    # print(f'_code = {_code}, code_display = {code_display }')
    docref.id = _id
    docref.date = _now
    if _subject == _subject:
        docref.subject.reference = _subject 
    if _author == _author:
        docref.author[0].reference = _author
        docref.author[0].display = _author.split('/')[-1].title()
    if _authenticator == _authenticator:
        docref.authenticator = dict(reference = _authenticator,
                                display = _authenticator.split('/')[-1].title()
                                )
        docref.authenticator.extension = [authentication_time_extension]

    if _category == _category:
        cat_display = get_display(_system,_category)
        # print(f'_category = {_category}, cat_display = {cat_display }')
        docref.category[0].coding[0].code = _category
        docref.category[0].coding[0].system = _system
        docref.category[0].coding[0].display = cat_display
        docref.category[0].text = cat_display
    else:
        docref.category = my_cat

    docref.type.coding[0].code = _code
    docref.type.coding[0].display = code_display
    docref.type.text = code_display.title()
    docref.content[0].attachment.title = _title
    try:
        docref.content[0].attachment.contentType = mimetypes.types_map[f".{_file.split('.')[-1]}"]
    except KeyError: #for "xlsx bug"
        docref.content[0].attachment.contentType = "application/vnd.ms-excel"
    docref.content[0].attachment.creation = _datetime
    docref.content[0].attachment.data = get_binary(_file)
    # print(f'docref.content[0].attachment.data = {type(docref.content[0].attachment.data)}')
    out_file = out_path / f'DocumentReference-{_id}.json'
    out_file.write_text(docref.json(indent=2))
    out_file = out_path / 'yaml' / f'DocumentReference-{_id}.yml'
    # change byte to string for base64
    docref_dict = docref.dict()
    docref_dict['content'][0]['attachment']['data'] = docref_dict['content'][0]['attachment']['data'].decode('utf-8')
    y_dump(docref_dict)
    out_file.write_text(y_dump(docref_dict, sort_keys=False))

0
Progress Note
200 OK
{'resourceType': 'Parameters', 'parameter': [{'name': 'code', 'valueCode': '11506-3'}, {'name': 'display', 'valueString': 'Progress note'}, {'name': 'name', 'valueString': 'LOINC 2.78'}, {'name': 'system', 'valueUri': 'http://loinc.org'}, {'name': 'version', 'valueString': '2.78'}, {'name': 'property', 'part': [{'name': 'code', 'valueCode': 'parent'}, {'name': 'value', 'valueCode': 'LP416685-8'}]}, {'name': 'property', 'part': [{'name': 'code', 'valueCode': 'inactive'}, {'name': 'value', 'valueBoolean': False}, {'name': 'valueBoolean', 'valueBoolean': False}]}, {'name': 'designation', 'part': [{'name': 'language', 'valueCode': 'es-MX'}, {'name': 'use', 'valueCoding': {'system': 'http://terminology.hl7.org/CodeSystem/hl7TermMaintInfra', 'code': 'preferredForLanguage', 'display': 'Preferred For Language'}}, {'name': 'value', 'valueString': 'Nota de progreso:{Configuración} :Punto temporal:Tipo:Documento:{Role}'}]}, {'name': 'designation', 'part': [{'name': 'languag